# Section 4: Experiment Tracking（実験管理）

## 🎯 このセクションの目標
- **Snowflake ML Experiments** を使った実験管理を学ぶ
- 複数モデルの比較実験を行う
- パラメータ・メトリクスを記録して追跡する

## 📋 Experiment Trackingとは？
機械学習の実験を体系的に管理するための機能です：
- モデルのパラメータと結果を記録
- 複数の実験を比較
- 再現性のある実験環境を構築

---

## 💡 ハンズオンでの位置づけ

このハンズオンでは、**各機能を独立して学ぶ**ことを目的としています。

| Section | 学習目的 |
|---------|----------|
| 03 Model Training | ハイパラ**自動探索**（RandomizedSearchCV） |
| 04 Experiment Tracking | 実験の**記録・比較**機能 |

そのため、03で見つけた最良パラメータとは**別に**、異なるパラメータセットで実験を行い、Experiment Trackingの機能を体験します。

> 📝 **実運用では**: RandomizedSearchCVの各イテレーション結果をExperiment Trackingに記録し、一元管理することが多いです。


---
## 4.1 環境設定

> 💡 **パッケージについて**: `environment.yml` により必要なパッケージ（snowflake-ml-python等）は自動的にインストールされています。


In [ ]:
# Warningを抑制（ハンズオン向け）
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# ML関連（sklearn版を使用 - コンテナランタイム互換性のため）
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix  # 🆕 追加
)

import pandas as pd
import numpy as np
import json
import time  # 🆕 実行時間計測用

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns  # 🆕 Confusion Matrix用

# SHAP（モデル解釈性）
import shap

# Experiment Tracking
from snowflake.ml.experiment import ExperimentTracking

# セッションの取得
session = get_active_session()

# 環境設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("FEATURE_STORE")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")


---
## 4.2 Experiment Trackingの初期化


In [ ]:
# ExperimentTracking のインスタンス化
exp = ExperimentTracking(session=session)

# 実験名を設定
EXPERIMENT_NAME = "CHURN_PREDICTION_EXPERIMENT"
exp.set_experiment(EXPERIMENT_NAME)

print(f"✅ Experiment設定完了: {EXPERIMENT_NAME}")


---
## 4.3 学習データの準備


In [ ]:
# 学習データセットを読み込み
DATASET_NAME = "TRAINING_DATASET_V1"  # 🆕 データセット名を変数化
training_data = session.table(DATASET_NAME)

# 特徴量とラベルの定義
FEATURE_COLS = [
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "ORDER_COUNT_2024_H1",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "TOTAL_AMOUNT_2024_H1",
    "RETURN_RATE"
]
LABEL_COL = "IS_CHURNED"
ID_COL = "CUSTOMER_ID"

# データの分割
model_data = training_data.select([ID_COL] + FEATURE_COLS + [LABEL_COL])
train_sf, test_sf = model_data.random_split(weights=[0.8, 0.2], seed=42)

# Pandas DataFrameに変換（sklearn用）
train_pd = train_sf.to_pandas()
test_pd = test_sf.to_pandas()

X_train = train_pd[FEATURE_COLS]
y_train = train_pd[LABEL_COL]
X_test = test_pd[FEATURE_COLS]
y_test = test_pd[LABEL_COL]

# 🆕 データセット情報の計算
total_samples = len(train_pd) + len(test_pd)
train_churn_rate = y_train.mean()
test_churn_rate = y_test.mean()

# データセット情報を辞書に保存（後でExperiment Trackingに記録）
dataset_info = {
    "dataset_name": DATASET_NAME,
    "total_samples": total_samples,
    "train_samples": len(train_pd),
    "test_samples": len(test_pd),
    "num_features": len(FEATURE_COLS),
    "train_churn_rate": float(train_churn_rate),
    "test_churn_rate": float(test_churn_rate)
}

print(f"📊 データセット情報")
print(f"   データセット: {DATASET_NAME}")
print(f"   訓練データ: {len(train_pd):,} 件 (チャーン率: {train_churn_rate:.1%})")
print(f"   テストデータ: {len(test_pd):,} 件 (チャーン率: {test_churn_rate:.1%})")
print(f"   特徴量数: {len(FEATURE_COLS)} 個")


---
## 4.4 実験の実行

複数のハイパーパラメータセットでモデルを訓練し、結果を記録します。

| Run | 特徴 | 期待 |
|-----|------|------|
| Baseline | デフォルトパラメータ | 基準モデル |
| DeepTree | 深い木 (max_depth=10) | 複雑なパターン学習 |
| Conservative | 浅い木 + 低学習率 | 過学習防止 |

### 📝 記録する情報

| 種類 | 内容 | 記録方法 |
|------|------|----------|
| パラメータ | ハイパーパラメータ | `log_param()` |
| データセット情報 | データ件数、チャーン率 | `log_param()` |
| メトリクス | Accuracy, F1, **ROC-AUC** | `log_metric()` |
| 実行時間 | 学習にかかった時間 | `log_metric()` |
| 特徴量重要度 | XGBoost Feature Importance | `log_metric()` |
| SHAP値 | 平均絶対SHAP値 | `log_metric()` |
| 画像 | Confusion Matrix, Feature Importance棒グラフ, SHAP Summary Plot | `log_artifact()` |


In [ ]:
# 実験設定の定義
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

experiments = [
    {
        "run_name": f"Baseline_{timestamp}",
        "display_name": "Baseline",
        "params": {
            "max_depth": 5,
            "learning_rate": 0.1,
            "n_estimators": 100,
            "scale_pos_weight": 2
        }
    },
    {
        "run_name": f"DeepTree_{timestamp}",
        "display_name": "DeepTree",
        "params": {
            "max_depth": 10,
            "learning_rate": 0.1,
            "n_estimators": 100,
            "scale_pos_weight": 2
        }
    },
    {
        "run_name": f"Conservative_{timestamp}",
        "display_name": "Conservative",
        "params": {
            "max_depth": 3,
            "learning_rate": 0.05,
            "n_estimators": 200,
            "scale_pos_weight": 3
        }
    }
]

print(f"✅ {len(experiments)}つの実験を定義しました")
print(f"   タイムスタンプ: {timestamp}")

In [ ]:
# 各実験の実行とログ記録
results = []

for experiment in experiments:
    run_name = experiment["run_name"]
    display_name = experiment["display_name"]
    params = experiment["params"]
    
    print(f"\n{'='*60}")
    print(f"Running: {display_name}")
    print(f"{'='*60}")
    
    # Experiment Trackingでランを開始
    with exp.start_run(run_name=run_name):
        
        # パラメータをログ
        exp.log_param("model_type", "XGBoost")
        exp.log_param("display_name", display_name)
        for param_name, param_value in params.items():
            exp.log_param(param_name, param_value)
        
        # 🆕 データセット情報をログ
        for key, value in dataset_info.items():
            exp.log_param(key, value)
        
        # 🆕 実行時間の計測開始
        start_time = time.time()
        
        # モデルの訓練（sklearn版XGBoost）
        model = xgb.XGBClassifier(
            random_state=42,
            use_label_encoder=False,
            eval_metric='logloss',
            **params
        )
        model.fit(X_train, y_train)
        
        # 🆕 実行時間の計測終了
        training_time = time.time() - start_time
        exp.log_metric("training_time_sec", training_time)
        
        # 予測と評価
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]  # 🆕 確率予測
        
        # メトリクス計算
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)  # 🆕 ROC-AUC
        
        # メトリクスをログ
        exp.log_metric("accuracy", accuracy)
        exp.log_metric("precision", precision)
        exp.log_metric("recall", recall)
        exp.log_metric("f1_score", f1)
        exp.log_metric("roc_auc", roc_auc)  # 🆕
        
        # =====================================================
        # 🆕 Confusion Matrix（混同行列）の計算と保存
        # =====================================================
        cm = confusion_matrix(y_test, y_pred)
        
        # Confusion Matrixをプロット
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Non-Churn', 'Churn'],
                    yticklabels=['Non-Churn', 'Churn'])
        plt.title(f'Confusion Matrix - {display_name}')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.tight_layout()
        
        # 画像を保存
        cm_plot_path = f"/tmp/confusion_matrix_{display_name}.png"
        plt.savefig(cm_plot_path, bbox_inches='tight', dpi=150)
        plt.close()
        
        # アーティファクトとして記録
        exp.log_artifact(cm_plot_path)
        
        # Confusion Matrixの数値も記録
        tn, fp, fn, tp = cm.ravel()
        exp.log_metric("true_negatives", int(tn))
        exp.log_metric("false_positives", int(fp))
        exp.log_metric("false_negatives", int(fn))
        exp.log_metric("true_positives", int(tp))
        
        print(f"\n  📊 Confusion Matrix:")
        print(f"    TN: {tn}, FP: {fp}")
        print(f"    FN: {fn}, TP: {tp}")
        
        # =====================================================
        # 特徴量重要度（Feature Importance）を記録
        # =====================================================
        feature_importance = model.feature_importances_
        importance_dict = dict(zip(FEATURE_COLS, [float(v) for v in feature_importance]))
        
        # 各特徴量の重要度を個別のメトリクスとして記録
        for feature_name, importance in importance_dict.items():
            exp.log_metric(f"importance_{feature_name}", importance)
        
        print(f"\n  📊 特徴量重要度 (XGBoost):")
        for feat, imp in sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)[:3]:
            print(f"    {feat}: {imp:.4f}")
        
        # 🆕 Feature Importance棒グラフを画像として保存
        sorted_imp = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)
        fig, ax = plt.subplots(figsize=(10, 6))
        features = [x[0] for x in sorted_imp]
        importances = [x[1] for x in sorted_imp]
        colors = plt.cm.Blues(np.linspace(0.8, 0.3, len(features)))
        ax.barh(features[::-1], importances[::-1], color=colors[::-1])
        ax.set_xlabel('Feature Importance')
        ax.set_title(f'Feature Importance - {display_name}', fontsize=14, fontweight='bold')
        for i, v in enumerate(importances[::-1]):
            ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)
        plt.tight_layout()
        
        fi_plot_path = f"/tmp/feature_importance_{display_name}.png"
        plt.savefig(fi_plot_path, bbox_inches='tight', dpi=150)
        plt.close()
        exp.log_artifact(fi_plot_path)
        
        # =====================================================
        # SHAP値の計算と記録
        # =====================================================
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)
        
        # SHAP値の平均絶対値（グローバル重要度）を計算・記録
        shap_importance = np.abs(shap_values).mean(axis=0)
        shap_dict = dict(zip(FEATURE_COLS, [float(v) for v in shap_importance]))
        
        for feature_name, shap_imp in shap_dict.items():
            exp.log_metric(f"shap_{feature_name}", shap_imp)
        
        print(f"\n  🔍 SHAP値 (平均絶対値):")
        for feat, shap_val in sorted(shap_dict.items(), key=lambda x: x[1], reverse=True)[:3]:
            print(f"    {feat}: {shap_val:.4f}")
        
        # 🆕 SHAP値の完全データをCSVとして保存（後から再現可能に）
        shap_full_df = pd.DataFrame(shap_values, columns=[f"shap_{col}" for col in FEATURE_COLS])
        # 特徴量の実際の値も追加（Summary Plotの色情報再現用）
        for col in FEATURE_COLS:
            shap_full_df[f"value_{col}"] = X_test[col].values
        shap_csv_path = f"/tmp/shap_full_data_{display_name}.csv"
        shap_full_df.to_csv(shap_csv_path, index=False)
        exp.log_artifact(shap_csv_path)
        
        # =====================================================
        # SHAP Summary Plotを画像として保存・アーティファクト記録
        # =====================================================
        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values, X_test, feature_names=FEATURE_COLS, show=False)
        plt.title(f"SHAP Summary Plot - {display_name}")
        plt.tight_layout()
        
        # 画像を保存
        shap_plot_path = f"/tmp/shap_summary_{display_name}.png"
        plt.savefig(shap_plot_path, bbox_inches='tight', dpi=150)
        plt.close()
        
        # アーティファクトとして記録
        exp.log_artifact(shap_plot_path)
        print(f"\n  📸 アーティファクト保存:")
        print(f"    - {cm_plot_path}")
        print(f"    - {fi_plot_path}")
        print(f"    - {shap_csv_path} (完全データ)")
        print(f"    - {shap_plot_path}")
        
        # 結果を保存
        results.append({
            "run_name": display_name,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "roc_auc": roc_auc,
            "training_time": training_time,
            "confusion_matrix": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
            "model": model,
            "params": params,
            "feature_importance": importance_dict,
            "shap_importance": shap_dict,
            "shap_values": shap_values
        })
        
        print(f"\n  ✅ メトリクス:")
        print(f"    Accuracy:  {accuracy:.4f}")
        print(f"    Precision: {precision:.4f}")
        print(f"    Recall:    {recall:.4f}")
        print(f"    F1 Score:  {f1:.4f}")
        print(f"    ROC-AUC:   {roc_auc:.4f}")
        print(f"    学習時間:  {training_time:.2f}秒")

print(f"\n✅ {len(experiments)}つの実験が完了しました")

---
## 4.5 実験結果の比較


In [ ]:
# 結果をDataFrameで比較
results_df = pd.DataFrame([{
    "Run": r["run_name"],
    "Accuracy": f"{r['accuracy']:.4f}",
    "Precision": f"{r['precision']:.4f}",
    "Recall": f"{r['recall']:.4f}",
    "F1 Score": f"{r['f1_score']:.4f}",
    "ROC-AUC": f"{r['roc_auc']:.4f}",
    "Time(s)": f"{r['training_time']:.2f}"
} for r in results])

print("=" * 90)
print("📊 実験結果の比較")
print("=" * 90)
print(results_df.to_string(index=False))
print("=" * 90)


In [ ]:
# 最良モデルの選択（F1スコア基準）
best_result = max(results, key=lambda x: x["f1_score"])

print("=" * 60)
print("🏆 最良モデル（F1スコア基準）")
print("=" * 60)
print(f"  Run Name:  {best_result['run_name']}")
print(f"  F1 Score:  {best_result['f1_score']:.4f}")
print(f"  ROC-AUC:   {best_result['roc_auc']:.4f}")
print(f"  Accuracy:  {best_result['accuracy']:.4f}")
print(f"  Precision: {best_result['precision']:.4f}")
print(f"  Recall:    {best_result['recall']:.4f}")
print(f"  学習時間:  {best_result['training_time']:.2f}秒")
print("="  * 60)
print(f"  パラメータ: {best_result['params']}")
print("=" * 60)

# Confusion Matrixの結果を表示
cm = best_result['confusion_matrix']
print(f"\n📊 Confusion Matrix:")
print(f"  True Negatives:  {cm['tn']:,}")
print(f"  False Positives: {cm['fp']:,}")
print(f"  False Negatives: {cm['fn']:,}")
print(f"  True Positives:  {cm['tp']:,}")

# 最良モデルの特徴量重要度を表示
print("\n📊 最良モデルの特徴量重要度（上位5件）:")
sorted_importance = sorted(best_result['feature_importance'].items(), key=lambda x: x[1], reverse=True)
for feat, imp in sorted_importance[:5]:
    print(f"  {feat}: {imp:.4f}")

print("\n🔍 最良モデルのSHAP値（上位5件）:")
sorted_shap = sorted(best_result['shap_importance'].items(), key=lambda x: x[1], reverse=True)
for feat, shap_val in sorted_shap[:5]:
    print(f"  {feat}: {shap_val:.4f}")

# 最良モデルを保存
best_model = best_result["model"]
best_metrics = {
    "accuracy": best_result["accuracy"],
    "precision": best_result["precision"],
    "recall": best_result["recall"],
    "f1_score": best_result["f1_score"],
    "roc_auc": best_result["roc_auc"]
}


---
## 4.6 可視化


In [ ]:
# 実験結果のバーチャート（メトリクス）
metrics_to_plot = ["accuracy", "precision", "recall", "f1_score", "roc_auc"]
run_names = [r["run_name"] for r in results]

fig, axes = plt.subplots(1, 5, figsize=(16, 4))

for i, metric in enumerate(metrics_to_plot):
    values = [r[metric] for r in results]
    colors = ['gold' if r["run_name"] == best_result["run_name"] else 'steelblue' for r in results]
    axes[i].bar(run_names, values, color=colors)
    axes[i].set_title(metric.replace('_', ' ').title())
    axes[i].set_ylim(0, 1)
    axes[i].tick_params(axis='x', rotation=45)

plt.suptitle('Experiment Results Comparison - Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()

# 🆕 メトリクス比較チャートをアーティファクトとして保存
metrics_comparison_path = f"/tmp/metrics_comparison_{timestamp}.png"
plt.savefig(metrics_comparison_path, bbox_inches='tight', dpi=150)
plt.show()

# 実行時間の比較
fig, ax = plt.subplots(figsize=(8, 4))
times = [r["training_time"] for r in results]
colors = ['gold' if r["run_name"] == best_result["run_name"] else 'steelblue' for r in results]
ax.bar(run_names, times, color=colors)
ax.set_title('Training Time Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Time (seconds)')
ax.tick_params(axis='x', rotation=45)
for i, t in enumerate(times):
    ax.annotate(f'{t:.2f}s', (i, t), ha='center', va='bottom')
plt.tight_layout()

# 🆕 実行時間比較チャートをアーティファクトとして保存
time_comparison_path = f"/tmp/training_time_comparison_{timestamp}.png"
plt.savefig(time_comparison_path, bbox_inches='tight', dpi=150)
plt.show()

# 🆕 実験全体の比較画像をアーティファクトとして記録
print("📸 実験比較チャートを保存しました:")
print(f"   - {metrics_comparison_path}")
print(f"   - {time_comparison_path}")


In [ ]:
# 最良モデルのSHAP Summary Plotを表示
print(f"=== 🔍 最良モデル（{best_result['run_name']}）のSHAP Summary Plot ===")

plt.figure(figsize=(10, 6))
shap.summary_plot(
    best_result['shap_values'], 
    X_test, 
    feature_names=FEATURE_COLS,
    show=False
)
plt.title(f"SHAP Summary Plot - {best_result['run_name']}", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 このプロットの見方:")
print("   - 横軸: SHAP値（予測への影響度）")
print("   - 縦軸: 特徴量（重要度順）")
print("   - 色: 特徴量の値（赤=高、青=低）")


---
## 4.7 実験結果の永続化


In [ ]:
# 実験結果をテーブルに保存（補助的な記録）
import datetime

experiment_records = []
for r in results:
    experiment_records.append({
        "EXPERIMENT_NAME": EXPERIMENT_NAME,
        "RUN_NAME": r["run_name"],
        "ACCURACY": r["accuracy"],
        "PRECISION": r["precision"],
        "RECALL": r["recall"],
        "F1_SCORE": r["f1_score"],
        "ROC_AUC": r["roc_auc"],                                    # 🆕 ROC-AUC
        "TRAINING_TIME_SEC": r["training_time"],                    # 🆕 実行時間
        "CONFUSION_MATRIX": json.dumps(r["confusion_matrix"]),      # 🆕 Confusion Matrix
        "PARAMS": json.dumps(r["params"]),
        "FEATURE_IMPORTANCE": json.dumps(r["feature_importance"]),
        "SHAP_IMPORTANCE": json.dumps(r["shap_importance"]),
        "DATASET_INFO": json.dumps(dataset_info),                   # 🆕 データセット情報
        "CREATED_AT": datetime.datetime.now().isoformat()
    })

records_df = session.create_dataframe(experiment_records)
records_df.write.save_as_table(
    "MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS",
    mode="overwrite"
)

print("✅ 実験結果をテーブルに保存しました")
print("   テーブル: MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS")
print("\n📝 保存された項目:")
print("   - メトリクス（Accuracy, Precision, Recall, F1, ROC-AUC）")
print("   - 実行時間")
print("   - Confusion Matrix（JSON）")
print("   - パラメータ（JSON）")
print("   - 特徴量重要度（JSON）")
print("   - SHAP値（JSON）")
print("   - データセット情報（JSON）")


---
## 4.7.1 Model Registryとの連携

最良モデルを **Experiment Tracking経由で** Model Registryに登録します。
これにより、**実験ラン ↔ モデルバージョン** がリンクされ、完全な追跡可能性（Traceability）を実現できます。

### 💡 MLOpsの本質
- 「このモデルはどの実験から来たのか？」が一目で分かる
- 実験のパラメータ・メトリクス・アーティファクトとモデルが紐付く


In [ ]:
# 最良モデルをModel Registryに登録（実験ランとリンク）
MODEL_NAME = "CUSTOMER_CHURN_PREDICTOR"
VERSION_NAME = "v1"

print(f"=== 🔗 最良モデルをModel Registryに登録 ===")
print(f"モデル名: {MODEL_NAME}")
print(f"バージョン: {VERSION_NAME}")
print(f"元の実験Run: {best_result['run_name']}")

# Model Registryに接続
from snowflake.ml.registry import Registry
registry = Registry(
    session=session,
    database_name="MLOPS_HOL_DB",
    schema_name="MODEL_REGISTRY"
)

# 既存のv1があれば削除（再実行対応）
try:
    model = registry.get_model(MODEL_NAME)
    versions = [v.version_name for v in model.versions()]
    if VERSION_NAME in versions:
        print(f"\n🔄 既存の {VERSION_NAME} を削除して再登録します...")
        model.delete_version(VERSION_NAME)
except:
    pass  # モデルが存在しない場合は何もしない

# 新しいランを開始し、その中でモデルを登録
best_model_run_name = f"BestModel_{timestamp}"

with exp.start_run(run_name=best_model_run_name):
    # 最良モデルの情報をログ
    exp.log_param("source_run", best_result['run_name'])
    exp.log_param("model_name", MODEL_NAME)
    exp.log_param("version", VERSION_NAME)
    
    # 最良モデルのパラメータを再ログ
    for param_name, param_value in best_result['params'].items():
        exp.log_param(param_name, param_value)
    
    # 最良モデルのメトリクスを再ログ
    exp.log_metric("accuracy", best_result['accuracy'])
    exp.log_metric("precision", best_result['precision'])
    exp.log_metric("recall", best_result['recall'])
    exp.log_metric("f1_score", best_result['f1_score'])
    exp.log_metric("roc_auc", best_result['roc_auc'])
    
    # 🔗 Model Registryに登録（実験ランとリンク）
    exp.log_model(
        model=best_model,
        model_name=MODEL_NAME,
        version_name=VERSION_NAME,
        sample_input_data=X_train.head(10),  # 入力形式のサンプル
        metrics=best_metrics,
        comment=f"""チャーン予測モデル - Experiment Tracking連携版

【Lineage（系譜）情報】
- 実験名: {EXPERIMENT_NAME}
- 元のRun: {best_result['run_name']}
- 登録Run: {best_model_run_name}

【パラメータ】
{best_result['params']}

【メトリクス】
- F1 Score: {best_result['f1_score']:.4f}
- ROC-AUC: {best_result['roc_auc']:.4f}

【特徴量（{len(FEATURE_COLS)}個）】
{', '.join(FEATURE_COLS)}"""
    )

print(f"\n✅ Model Registryに登録完了！")
print(f"   モデル: {MODEL_NAME}")
print(f"   バージョン: {VERSION_NAME}")
print(f"   リンクされたRun: {best_model_run_name}")
print(f"\n🔗 Snowsight → AI & ML → Experiments で確認すると、")
print(f"   「モデルバージョン」にリンクが表示されます！")


---
## 4.8 後から振り返る方法（アーティファクトの取り出し）

Experiment Trackingに保存したアーティファクト（画像やCSV）は、後からノートブックで取り出せます。

### 💡 ユースケース
- 「3ヶ月前のモデル、なぜこの予測をしたんだっけ？」
- 「過去の実験のSHAP値を再分析したい」
- 「監査のために当時のモデル解釈情報を提出したい」


In [ ]:
# 過去の実験からアーティファクトを取り出す例
# ※ 実際に振り返る際は、run_name を過去の実験名に変更してください

# 今回の実験のrun_nameを使用（例として最初のBaseline）
example_run_name = experiments[0]["run_name"]

print(f"=== 📂 アーティファクト一覧の確認 ===")
print(f"Run: {example_run_name}\n")

# アーティファクト一覧を取得
artifacts = exp.list_artifacts(run_name=example_run_name)
print("保存されているアーティファクト:")
for artifact in artifacts:
    print(f"  - {artifact}")


In [ ]:
# SHAP完全データをダウンロードしてSummary Plotを再現
print(f"=== 🔍 SHAP Summary Plotの再現 ===")
print(f"Run: {example_run_name}\n")

# アーティファクトをダウンロード
shap_artifact_name = "shap_full_data_Baseline.csv"
local_download_path = "/tmp/downloaded_shap/"

import os
os.makedirs(local_download_path, exist_ok=True)

# ※ メソッド名は download_artifacts（複数形）、引数は target_path
exp.download_artifacts(
    run_name=example_run_name,
    artifact_path=shap_artifact_name,
    target_path=local_download_path
)

print(f"✅ ダウンロード完了: {local_download_path}{shap_artifact_name}")

# CSVを読み込み
downloaded_shap_df = pd.read_csv(f"{local_download_path}{shap_artifact_name}")
print(f"   データ形状: {downloaded_shap_df.shape}")

# SHAP値と特徴量値を分離
shap_cols = [f"shap_{col}" for col in FEATURE_COLS]
value_cols = [f"value_{col}" for col in FEATURE_COLS]

downloaded_shap_values = downloaded_shap_df[shap_cols].values
downloaded_feature_values = downloaded_shap_df[value_cols].values

# Summary Plotを再現！
print("\n📊 過去の実験のSHAP Summary Plotを再現:")
plt.figure(figsize=(10, 6))
shap.summary_plot(
    downloaded_shap_values, 
    downloaded_feature_values, 
    feature_names=FEATURE_COLS,
    show=False
)
plt.title(f"SHAP Summary Plot (再現) - {example_run_name}", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ 過去の実験データから完全に再現できました！")


---
## ✅ Section 4 完了！

### 学んだこと
- **Snowflake ML ExperimentTracking** の使い方
- `exp.set_experiment()` で実験を作成
- `exp.start_run()` でランを開始
- `exp.log_param()`, `exp.log_metric()` でパラメータ・メトリクスを記録
- 複数実験の比較と最良モデルの選択

### 📝 記録した情報

| メソッド | 記録内容 | 用途 |
|---------|---------|------|
| `log_param()` | ハイパーパラメータ | モデル設定の追跡 |
| `log_param()` | データセット情報 | 再現性の確保 |
| `log_metric()` | Accuracy, Precision, Recall, F1 | 分類性能評価 |
| `log_metric()` | ROC-AUC | 閾値に依存しない性能評価 |
| `log_metric()` | 実行時間 | パフォーマンス比較 |
| `log_metric()` | Confusion Matrix (TN/FP/FN/TP) | 誤分類パターン分析 |
| `log_metric()` | 特徴量重要度 | モデル解釈性 |
| `log_metric()` | SHAP値 | 予測根拠の説明 |
| `log_artifact()` | Confusion Matrix画像 | 誤分類パターンの可視化 |
| `log_artifact()` | Feature Importance棒グラフ | 特徴量重要度の可視化 |
| `log_artifact()` | **SHAP完全データ（CSV）** | **後からSummary Plot再現** |
| `log_artifact()` | SHAP Summary Plot | SHAP値の可視化 |

### 作成したもの
- Experiment: `CHURN_PREDICTION_EXPERIMENT`
- 3つのRun: Baseline, DeepTree, Conservative
- 実験結果テーブル: `EXPERIMENT_RESULTS`
  - メトリクス（Accuracy, Precision, Recall, F1, ROC-AUC）
  - 実行時間
  - Confusion Matrix（JSON）
  - パラメータ（JSON）
  - 特徴量重要度（JSON）
  - SHAP値（JSON）
  - データセット情報（JSON）
- アーティファクト（各ランに4ファイルずつ）
  - `confusion_matrix_{Run名}.png` - 混同行列
  - `feature_importance_{Run名}.png` - 特徴量重要度棒グラフ
  - `shap_full_data_{Run名}.csv` - **SHAP完全データ（再現用）**
  - `shap_summary_{Run名}.png` - SHAP Summary Plot

### 💡 後から振り返るときのポイント

**方法1: SQLでメトリクス・パラメータを確認**
```sql
SELECT 
    RUN_NAME, F1_SCORE, ROC_AUC, TRAINING_TIME_SEC,
    FEATURE_IMPORTANCE, SHAP_IMPORTANCE
FROM MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS
ORDER BY F1_SCORE DESC;
```

**方法2: Pythonでアーティファクトを取り出してSHAP再現**
```python
# アーティファクト一覧
artifacts = exp.list_artifacts(run_name="過去のrun_name")

# SHAP完全データをダウンロード（※ download_artifacts は複数形）
exp.download_artifacts(run_name="過去のrun_name", 
                       artifact_path="shap_full_data_Baseline.csv",
                       target_path="/tmp/")

# Summary Plotを再現
shap_df = pd.read_csv("/tmp/shap_full_data_Baseline.csv")
shap.summary_plot(shap_df[shap_cols].values, shap_df[value_cols].values)
```

### 次のステップ
**Section 5: Model Registry** では、最良モデルを登録し、
バージョン管理と本番デプロイの準備を行います。

➡️ `05_MODEL_REGISTRY` Notebookに進んでください
